In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ab-testing-dataset/test_group.csv
/kaggle/input/ab-testing-dataset/control_group.csv


In [76]:
import pandas as pd
import numpy as np
from scipy import stats



In [77]:
# Load the datasets from the provided file paths
control_group = pd.read_csv('/kaggle/input/ab-testing-dataset/control_group.csv', sep=";")
test_group = pd.read_csv('/kaggle/input/ab-testing-dataset/test_group.csv', sep=";")

In [78]:
# Merge the dataframes
df = pd.concat([control_group, test_group], ignore_index=True)

# Display the first few rows of the merged dataframe
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Campaign Name        60 non-null     object 
 1   Date                 60 non-null     object 
 2   Spend [USD]          60 non-null     int64  
 3   # of Impressions     59 non-null     float64
 4   Reach                59 non-null     float64
 5   # of Website Clicks  59 non-null     float64
 6   # of Searches        59 non-null     float64
 7   # of View Content    59 non-null     float64
 8   # of Add to Cart     59 non-null     float64
 9   # of Purchase        59 non-null     float64
dtypes: float64(7), int64(1), object(2)
memory usage: 4.8+ KB


In [81]:
# Step 1: Data Preparation
# Ensure that the dataset is clean and ready for analysis

# Check for missing values
print(df.isnull().sum())

# Fill or drop missing values if necessary
df = df.dropna()


Campaign Name          0
Date                   0
Spend [USD]            0
# of Impressions       0
Reach                  0
# of Website Clicks    0
# of Searches          0
# of View Content      0
# of Add to Cart       0
# of Purchase          0
dtype: int64


In [82]:
df.columns

Index(['Campaign Name', 'Date', 'Spend [USD]', '# of Impressions', 'Reach',
       '# of Website Clicks', '# of Searches', '# of View Content',
       '# of Add to Cart', '# of Purchase'],
      dtype='object')

In [83]:
# Convert columns to appropriate data types
df['Campaign Name'] = df['Campaign Name'].astype('category')
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
df['Spend [USD]'] = df['Spend [USD]'].astype(float)
df['# of Impressions'] = df['# of Impressions'].astype(int)
df['Reach'] = df['Reach'].astype(int)
df['# of Website Clicks'] = df['# of Website Clicks'].astype(int)
df['# of Searches'] = df['# of Searches'].astype(int)
df['# of View Content'] = df['# of View Content'].astype(int)
df['# of Add to Cart'] = df['# of Add to Cart'].astype(int)
df['# of Purchase'] = df['# of Purchase'].astype(int)

In [84]:
df.describe()

,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
count,59,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,2019-08-15 16:16:16.271186432,2435.762712,91775.881356,70868.644068,5682.593220,2321.813559,1900.169492,1087.220339,522.000000
min,2019-08-01 00:00:00,1757.000000,22521.000000,10598.000000,2277.000000,1001.000000,848.000000,278.000000,222.000000
25%,2019-08-08 12:00:00,2113.500000,68853.500000,43230.000000,4227.500000,1962.500000,1249.000000,848.500000,340.000000
50%,2019-08-16 00:00:00,2421.000000,95843.000000,76219.000000,5691.000000,2390.000000,1975.000000,1075.000000,501.000000
75%,2019-08-23 00:00:00,2743.000000,117315.000000,95490.500000,7218.500000,2778.500000,2424.000000,1386.500000,693.000000
max,2019-08-30 00:00:00,3112.000000,145248.000000,127852.000000,8264.000000,4891.000000,4219.000000,1913.000000,890.000000
std,NaN,376.358795,32465.070636,31023.653515,1754.777559,669.040576,687.263844,430.205246,196.973917


In [85]:
# Step 2: Define Control and Test Groups
control_group = df[df['Campaign Name'] == 'Control Campaign']
test_group = df[df['Campaign Name'] == 'Test Campaign']

# Step 3: Perform Multiple A/B Tests for All Appropriate Columns
metrics = ['Spend [USD]', '# of Impressions', 'Reach', '# of Website Clicks', '# of Searches', '# of View Content', '# of Add to Cart', '# of Purchase']

results = {}

for metric in metrics:
    control_metric = control_group[metric]
    test_metric = test_group[metric]
    
    t_stat, p_value = stats.ttest_ind(control_metric, test_metric)
    
    results[metric] = {
        'T-statistic': t_stat,
        'P-value': p_value,
        'Significant': p_value < 0.05
    }

# Display the results
for metric, result in results.items():
    print(f"\nMetric: {metric}")
    print(f"T-statistic: {result['T-statistic']}")
    print(f"P-value: {result['P-value']}")
    if result['Significant']:
        print("Reject the null hypothesis. There is a significant difference between the control and test groups.")
    else:
        print("Fail to reject the null hypothesis. There is no significant difference between the control and test groups.")


Metric: Spend [USD]
T-statistic: -2.793240375594836
P-value: 0.007093062303374369
Reject the null hypothesis. There is a significant difference between the control and test groups.

Metric: # of Impressions
T-statistic: 4.884544325740239
P-value: 8.774394114329258e-06
Reject the null hypothesis. There is a significant difference between the control and test groups.

Metric: Reach
T-statistic: 5.300379386681559
P-value: 1.946208181590047e-06
Reject the null hypothesis. There is a significant difference between the control and test groups.

Metric: # of Website Clicks
T-statistic: -1.576909404840952
P-value: 0.12035072366063822
Fail to reject the null hypothesis. There is no significant difference between the control and test groups.

Metric: # of Searches
T-statistic: -1.1373340684043094
P-value: 0.26015715752487034
Fail to reject the null hypothesis. There is no significant difference between the control and test groups.

Metric: # of View Content
T-statistic: 0.47615455602474466
P-va

# Summary

- **Significant Differences**:
  - **Spending [USD]**: Control group spent more.
  - **Number of Impressions**: Test group had more impressions.
  - **Reach**: Test group had higher reach.
  - **Number of Adds to Cart**: Test group had more adds to cart.

- **No Significant Differences**:
  - **Number of Website Clicks**
  - **Number of Searches**
  - **Number of Content Views**
  - **Number of Purchases**

# Conclusion

## The A/B testing results indicate significant differences between the control and test groups for the following metrics:

- **Spend [USD]**: The test group spent less on average.
- **Number of Impressions**: The test group had more impressions.
- **Reach**: The test group had a higher reach.
- **Number of Add to Cart**: The test group had more adds to cart.

For the other metrics (website clicks, searches, view content, and purchases), there were no significant differences between the control and test groups. This suggests that while the test campaign was more effective in terms of impressions, reach, and adds to cart, it did not significantly impact website clicks, searches, content views, or purchases. The test group did reduce spending, therefore there should be consideration on whether the benefits seen outweigh the costs. 